In [2]:
def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime
        
path_to_info = dict()        
for f in os.listdir():
    path_to_info[f] = creation_date(f)
    
pprint(path_to_info)

NameError: name 'pprint' is not defined

In [3]:
import os
import platform
import tarfile

class BucketArchive():
    def __init__(self, filename, path=os.getcwd()):
        self.filename_fq = '{}/{}'.format(path, filename)
        self.last_modified_time = creation_date(filename)
        self.expanded = False
    
    def expand(self):
        collect = []
        if self.expanded == False:
            if tarfile.is_tarfile(self.filename_fq):
                opened_tar = tarfile.open(self.filename_fq)
                for member in opened_tar:
                    collect.append(member)
        return collect
    
def main():    
    ba = BucketArchive('./arXiv_src_1510_012.tar')
    expansion = ba.expand()
    for elt in expansion: print(elt)
        
main()


<TarInfo '1510' at 0x10b13b110>
<TarInfo '1510/1510.06261.gz' at 0x10b13b1d8>
<TarInfo '1510/1510.06039.gz' at 0x10b13b2a0>
<TarInfo '1510/1510.06302.gz' at 0x10b13b368>
<TarInfo '1510/1510.05982.gz' at 0x10b13b430>
<TarInfo '1510/1510.06037.gz' at 0x10b13b4f8>
<TarInfo '1510/1510.06180.gz' at 0x10b13b5c0>
<TarInfo '1510/1510.06285.gz' at 0x10b13b688>
<TarInfo '1510/1510.05995.gz' at 0x10b13b750>
<TarInfo '1510/1510.06204.gz' at 0x10b13b818>
<TarInfo '1510/1510.06313.gz' at 0x10b13b8e0>
<TarInfo '1510/1510.05931.pdf' at 0x10b13b9a8>
<TarInfo '1510/1510.05938.gz' at 0x10b13ba70>
<TarInfo '1510/1510.05874.pdf' at 0x10b13bb38>
<TarInfo '1510/1510.06197.gz' at 0x10b13bc00>
<TarInfo '1510/1510.06103.gz' at 0x10b13bcc8>
<TarInfo '1510/1510.05847.gz' at 0x10b13bd90>
<TarInfo '1510/1510.06314.gz' at 0x10b146048>
<TarInfo '1510/1510.05893.gz' at 0x10b146110>
<TarInfo '1510/1510.06064.gz' at 0x10b1461d8>
<TarInfo '1510/1510.06317.gz' at 0x10b1462a0>
<TarInfo '1510/1510.05921.gz' at 0x10b146368>


In [5]:
tars = []
for item in os.listdir():
    l = len(item)
    if item[l-4:l] == ".tar":
        tars.append(item)

print(len(os.listdir()))
print(len(tars))

2030
2008


In [11]:
class CharTree(object):
    class CharNode:
        def __init__(self, ch):
            self.char = ch
            self.children = {}
        def just_chars(self, yn):
            if yn:
                return list(self.children.keys())
            else:
                return self.children
            
        def children(self, justChars=False):
            return just_chars(self, justChars)
        def char(self):
            return self.char
        
        def __private_descendent_info(ch):
            return {'ch': ch, 'rf': CharNode(ch), 'on': True, '#': 1}
        
        def __private_merge_maps(map1, map2):
            items_in_both = set(map1.keys()).intersection(set(map2.keys()))   # all of these need to have their 'n' values updates in map1
            items_in_2_not_in_1 = set(map2.keys()).difference(items_in_both)            
            
            for i1 in items_in_2_not_in_1:
                map1[i1] = map2[i1]
            
            for i2 in items_in_both:
                payload = map1[i2]
                if payload['#'] is None:
                    payload['#'] = 1
                elif payload['#'] >= 1:
                    payload['#'] += 1
                else:
                    # will never get here 
                    0
                map1[i2] = payload
            return map1

        def add_descendents(self, chs):
            new_descendents = {}
            for ch in chs:
                new_descendents[ch] = __private_descendent_info(ch)
            if self.children is None or len(self.children) == 0:
                self.children = new_descendents
            else:
                existing_kids = self.children
                __private_merge_maps(existing_kids, new_descendents)
                    
            
            
        def forget_descendents(self, chs):
            if self.children is None or len(self.children) == 0:
                print("Node has no children.")
                print("Why did you call this method?")
                print("This has been logged.")
                self.children = {}
            else:
                for ch in chs:
                    for (k,v) in self.children.items():
                        if v['ch'] is ch:
                            v['on'] = False
                
                
    def __init__(self):
        self.root = None
        
    def generate_tree(self, docs=['This is an example', 'Example needs some interesting text', 'Boring!!!']):
        self.root = CharNode("")
        self.docs = docs
        